## 104 job web crawler

In [1]:
import re
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [2]:
#透過網址修改頁數
def get_url(page):
    template = "https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=python&expansionType=area%2Cspec%2Ccom%1Cjob%2Cwf%2Cwktm&area=6001001000&order=12&asc=0&page={}&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0"
    url = template.format(page)
    return url

In [3]:
url = get_url("1")

## parsing  single one

In [4]:
response = requests.get(url)

In [5]:
response

<Response [200]>

In [6]:
response.reason

'OK'

In [7]:
soup = BeautifulSoup(response.text, 'html.parser')

In [8]:
bords = soup.find_all('article', 'b-block--top-bord')

In [9]:
len(bords)

24

In [10]:
 bord = bords[8]

In [11]:
atag = bord.h2.a

In [12]:
type(atag.text)

str

In [13]:
job_title = bord.h2.a.text
job_title

'Python開發'

In [14]:
company_name =  bord.ul.find('li',{'class':False}).a.text.strip()
company_name

'英屬維京群島商文思海輝技術有限公司台灣分公司'

In [15]:
many_info = bord.find('ul', {'class':"b-list-inline b-clearfix job-list-intro b-content"}).text.strip().split()

In [16]:
degree = many_info.pop()
experience = many_info.pop()
location = many_info.pop()

In [17]:
job_summary =  bord.find('p').text.strip().replace("\n","")

In [18]:
salary_tag = bord.find('div', {'class':'job-list-tag b-content'}).span.text
if salary_tag == '待遇面議':
    salary = ''
else:
    salary = salary_tag

In [19]:
job_url = 'https:' + bord.h2.a.get("href")

In [20]:
date = bord.find('span',{'classs', 'b-tit__date'}).text.strip().replace('\n','')
if date == '':
    date = 'sys recommend'
today = datetime.today().strftime('%Y-%m-%d')

In [21]:
record = (job_title, company_name, location, experience, degree, job_summary, salary, job_url, date, today)

In [22]:
record

('Python開發',
 '英屬維京群島商文思海輝技術有限公司台灣分公司',
 '台北市中正區',
 '經歷不拘',
 '專科',
 '1. 分析客戶需求，程序設計/框架流程等2. 按照項目計劃，完成相應的任務3. 輔助工具開發和維護Job Requirements：1. 熟練掌握 Python語言，2年以上python相關工作經驗；2. 有使用 Linux 的經驗；',
 '月薪50,000~80,000元',
 'https://www.104.com.tw/job/7gel0?jobsource=jolist_b_relevance',
 '1/06',
 '2022-01-11')

## put all to function

In [23]:
def get_record(bord):
    job_title = bord.h2.a.text
    company_name =  bord.ul.find('li',{'class':False}).a.text.strip()
    job_summary =  bord.find('p').text.strip().replace("\n","").replace("\r","")
    job_url = 'https:' + bord.h2.a.get("href")
    today = datetime.today().strftime('%Y-%m-%d')    
    
    #多個<li>內容
    many_info = bord.find('ul', {'class':"b-list-inline b-clearfix job-list-intro b-content"}).text.strip().split()
    degree = many_info.pop()
    experience = many_info.pop()
    location = many_info.pop()
    
    #薪資面議
    salary_tag = bord.find('div', {'class':'job-list-tag b-content'}).span.text
    if salary_tag == '待遇面議':
        salary = ''
    else:
        salary = salary_tag
    
    #系統推薦無日期
    date = bord.find('span',{'classs', 'b-tit__date'}).text.strip().replace('\n','')
    if date == '':
        date = 'system recommend'
    
    record = (job_title, company_name, location, experience, degree, job_summary, salary, job_url, date, today)    
    return record

In [24]:
records = []

for bord in bords:
    record = get_record(bord)
    records.append(record)

In [25]:
records[0]

('【運維】網路管理工程師(不同班別各1至2名)',
 '帝群網路科技有限公司',
 '台北市內湖區',
 '經歷不拘',
 '專科',
 '工作職責：1、依專案需求，建置 AWS、Aliyun等雲產品，進行環境配置、上線部署、安全管控等工作。2、負責網路環境管理及監控，包括端口開放管理、漏洞與補丁管理、防火牆策略設定。3、負責日常服務器的巡檢，包括狀態監控、升級、故障處理及優化等操作。4、負責及時回應生產環境的監控預警、故障及突發情況的分析和解決。5、負責運維相關腳本開發和平臺部署和自動化管理。6、主管交辦事項。任職要求：1、熟悉 TCP/IP 和 HTTP 協議及相關場景，能排查網路障礙，優化網路質量。2、有 Web 系統和相關組件的維護經驗，並曾佈署及維運Nginx等網站服務。3、熟悉腳本語言 Shell / Python 並有相關開發應用經驗，能進行系統診斷和優化。4、熟悉常用的監控系統，如 Zabbix、Prometheus、Cacti 等。5、擁有1年以上 linux 系統使用和管理經驗，熟悉系統原理。',
 '',
 'https://www.104.com.tw/job/7am7x?jobsource=hotjob_chr',
 'system recommend',
 '2022-01-11')

## 10 pages data

In [27]:
for page in range(1,11):
    url = get_url(str(page))
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    bords = soup.find_all('article', 'b-block--top-bord')
    
    for bord in bords:
        record = get_record(bord)
        records.append(record)
len(records)

208

## integration

In [29]:
import re
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

#透過網址修改頁數
def get_url(page):
    template = "https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=python&expansionType=area%2Cspec%2Ccom%1Cjob%2Cwf%2Cwktm&area=6001001000&order=12&asc=0&page={}&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0"
    url = template.format(page)
    return url

#拿資料
def get_record(bord):
    job_title = bord.h2.a.text
    company_name =  bord.ul.find('li',{'class':False}).a.text.strip()
    job_summary =  bord.find('p').text.strip().replace("\n","").replace("\r","")
    job_url = 'https:' + bord.h2.a.get("href")
    today = datetime.today().strftime('%Y-%m-%d')    
    
    #多個<li>內容
    many_info = bord.find('ul', {'class':"b-list-inline b-clearfix job-list-intro b-content"}).text.strip().split()
    degree = many_info.pop()
    experience = many_info.pop()
    location = many_info.pop()
    
    #薪資面議
    salary_tag = bord.find('div', {'class':'job-list-tag b-content'}).span.text
    if salary_tag == '待遇面議':
        salary = ''
    else:
        salary = salary_tag
    
    #系統推薦無日期
    date = bord.find('span',{'classs', 'b-tit__date'}).text.strip().replace('\n','')
    if date == '':
        date = 'system recommend'
    
    record = (job_title, company_name, location, experience, degree, job_summary, salary, job_url, date, today)    
    return record

def main(page):
    record = []
    
    #get data
    for page in range(1,page):
        url = get_url(str(page))

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        bords = soup.find_all('article', 'b-block--top-bord')
    
        for bord in bords:
            record = get_record(bord)
            records.append(record)
            
    #save data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'Experience', 'Degree', 'Summary', 'Salary', 'JobUrl','PostDate', 'ExtractDate'  ])
        writer.writerows(records)

In [30]:
#主程式
main(11)

In [34]:
import pandas as pd

df = pd.read_csv('./results.csv')
df.head(10)

,JobTitle,Company,Location,Experience,Degree,Summary,Salary,JobUrl,PostDate,ExtractDate
0,【運維】網路管理工程師(不同班別各1至2名),帝群網路科技有限公司,台北市內湖區,經歷不拘,專科,工作職責：1、依專案需求，建置 AWS、Aliyun等雲產品，進行環境配置、上線部署、安全管...,NaN,https://www.104.com.tw/job/7am7x?jobsource=hot...,system recommend,2022-01-11
1,網路行銷／數位行銷,猿創力程式設計學校_壞主意有限公司,台北市松山區,經歷不拘,專科,【關於網路行銷／數位行銷】網路行銷／數位行銷部，是猿創力的佈道師和化粧師，藉由強大的創意和行...,"月薪30,000~40,000元",https://www.104.com.tw/job/6c59o?jobsource=hot...,system recommend,2022-01-11
2,【儲備幹部】財金儲備幹部(TA),玉山銀行_玉山商業銀行股份有限公司,台北市松山區,經歷不拘,碩士,直接走進一流交易室，站上國際金融舞台1.接受輪調培育訓練，建立全面Treasury專業視野2...,NaN,https://www.104.com.tw/job/6v672?jobsource=hot...,system recommend,2022-01-11
3,Database/Back-end Engineer 後端資料庫開發工程師(可遠距上班),美商迪諾亞股份有限公司台灣分公司,台北市內湖區,3年以上,大學,本職缺為後端開發工程師 (Back-end Engineer)， 從事本公司雲端服務器的後端...,NaN,https://www.104.com.tw/job/7hk4b?jobsource=hot...,system recommend,2022-01-11
4,Python Engineer,億力資訊股份有限公司,台北市內湖區,經歷不拘,高中,1.具備Python實務專案經驗2.有Vue開發框架佳3.太陽能系統開發4.主動性強、樂與團...,NaN,https://www.104.com.tw/job/5z6t9?jobsource=jol...,1/06,2022-01-11
5,Python Web Developer,StreetVoice _街聲股份有限公司,台北市松山區,1年以上,大學,開發派歌網站系統，主要針對音樂人以及業務管理端的使用者介面、流程進行開發及修改，持續優化。工...,"月薪40,000~60,000元",https://www.104.com.tw/job/7982k?jobsource=jol...,12/30,2022-01-11
6,Python工程師,雙子數位科技有限公司,台北市內湖區,經歷不拘,學歷不拘,嗨～～哈囉～你是本公司要找的後端工程師高手嗎?就是你！別懷疑！！徵的就是你～本公司需要有在P...,NaN,https://www.104.com.tw/job/7g51e?jobsource=jol...,1/05,2022-01-11
7,python工程師,英特拉金融科技股份有限公司,台北市大安區,3年以上,大學,工作內容： 1. Python維運及系統開發2.熟悉Python語言3.既有程式優化及修正並...,NaN,https://www.104.com.tw/job/7fbet?jobsource=jol...,1/10,2022-01-11
8,Python開發,英屬維京群島商文思海輝技術有限公司台灣分公司,台北市中正區,經歷不拘,專科,1. 分析客戶需求，程序設計/框架流程等2. 按照項目計劃，完成相應的任務3. 輔助工具開發...,"月薪50,000~80,000元",https://www.104.com.tw/job/7gel0?jobsource=jol...,1/06,2022-01-11
9,Python Developer / Senior Python Developer,鏈科股份有限公司,台北市松山區,3年以上,大學,government (NDF)!We’re hiring for python devel...,"年薪700,000~1,500,000元",https://www.104.com.tw/job/7ff0w?jobsource=jol...,1/10,2022-01-11
